<br>

<h1 style="text-align:center;">Text generation with a miniature GPT</h1>

<br>

### Introduction

---

Implementation of an autoregressive language model using the GPT model.
 
We use the IMDB sentiment classification dataset for training generate new movie reviews for a given prompt.

[GPT](https://www.semanticscholar.org/paper/Improving-Language-Understanding-by-Generative-Radford/cd18800a0fe0b668a1cc19f2ec95b5003d0a5035),
[GPT-2](https://www.semanticscholar.org/paper/Language-Models-are-Unsupervised-Multitask-Learners-Radford-Wu/9405cc0d6169988371b2755e573cc28650d14dfe),
[GPT-3](https://arxiv.org/abs/2005.14165)

<br>

### INITIAL SETUP

---

In [1]:
# Import the libraries
import re, os, string, random
import numpy as np
import tensorflow as tf

In [2]:
tf.range(1, 3)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([1, 2])>

<br>

### TRANSFORMER BLOCK

---

In [3]:
# Causal attention mask function
def causal_attention_mask(batch_size, n_dest, n_src, dtype):
    """
    This function creates a causal attention mask for the transformer model.
    More specifically, it will mask the upper half of the dot product matrix in 
    self attention (to prevent flow of information from future tokens to current).
    ARGUMENTS
    =================
        - batch_size: batch size of the input
        - n_dest: number of tokens in the destination sequence
        - n_src: number of tokens in the source sequence
        - dtype: data type of the mask
        
    RETURNS
    =================
        - out: causal attention mask
    """
    
    # Initialize the indices
    i = tf.range(n_dest)[:, None]
    j = tf.range(n_src)
    
    # Create the mask 
    m = i >= j - n_src + n_dest
    
    # Cast the mask
    mask = tf.cast(m, dtype)
    
    # Expand the mask
    mask = tf.reshape(mask, [1, n_dest, n_src])
    
    # This is the mask that we will use to mask the upper half of the dot product matrix
    mult = tf.concat([tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], 0)
    
    # Tile the mask 
    out = tf.tile(mask, mult)
    
    return out

In [4]:
# Transformer block class
class TransformerBlock(tf.keras.layers.Layer):

    # Constructo function
    def __init__(self, embed_dim: int, num_heads: int, ff_dim: int, rate: float=0.1):

        # Inherite parent's constructor
        super(TransformerBlock, self).__init__()

        # Self-attention layer
        self.att = tf.keras.layers.MultiHeadAttention(num_heads, embed_dim)

        # Feed forward network
        self.ffn = tf.keras.Sequential(
            [tf.keras.layers.Dense(ff_dim, activation="relu"), tf.keras.layers.Dense(embed_dim),]
        )

        # Layer normalizations
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        
        # Dropouts
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    # Call function
    def call(self, inputs: tf.Tensor, training: bool = False) -> tf.Tensor:
        
        # Initialize variables
        input_shape = tf.shape(inputs)
        batch_size = input_shape[0]
        seq_len = input_shape[1]
        causal_mask = causal_attention_mask(batch_size, seq_len, seq_len, tf.bool)
        
        # First attention block
        attention_output = self.att(inputs, inputs, attention_mask=causal_mask)
        attention_output = self.dropout1(attention_output, training=training)
        out1 = self.layernorm1(inputs + attention_output)
        
        # Second attention block
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        out = self.layernorm2(out1 + ffn_output)
        
        return out

<br>

### EMBEDDING LAYER

---

Create two seperate embedding layers: one for tokens and one for token index
(positions).

In [5]:
# Token and position embedding class
class TokenAndPositionEmbedding(tf.keras.layers.Layer):
    
    # Initialize the constructor
    def __init__(self, maxlen, vocab_size, embed_dim):
        
        # Inherit the parent's constructor
        super(TokenAndPositionEmbedding, self).__init__()
        
        # Token embedding layer
        self.token_emb = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        
        # Position embedding layer
        self.pos_emb = tf.keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    # Call function
    def call(self, x):
        
        # Maximum sequence length
        maxlen = tf.shape(x)[-1]
        
        # Initialize the positions
        positions = tf.range(start=0, limit=maxlen, delta=1)
        
        # Feed the positions to the position embedding layer
        positions = self.pos_emb(positions)
        
        # Feed the tokens to the token embedding layer
        x = self.token_emb(x)
        
        # Add the token and position embeddings
        out = x + positions
        
        return out

<br>

### GPT model

---

In [6]:
# Initialization
vocab_size = 20000       # Vocabulary (only consider the top 20k words)
maxlen = 80              # Max sequence size
embed_dim = 256          # Embedding size for each token
num_heads = 2            # Number of attention heads
feed_forward_dim = 256   # Hidden layer size in feed forward network inside transformer

In [7]:
# GPT model
def create_model():
    
    # Input layer
    inputs = tf.keras.layers.Input(shape=(maxlen,), dtype=tf.int32)
    
    # Token and position embedding layer
    embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
    x = embedding_layer(inputs)
    
    # Transformer block
    transformer_block = TransformerBlock(embed_dim, num_heads, feed_forward_dim)
    x = transformer_block(x)
    
    # Output layer
    outputs = tf.keras.layers.Dense(vocab_size)(x)
    
    # Construct the model
    model = tf.keras.Model(inputs=inputs, outputs=[outputs, x])
    
    # Loss function
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    
    # Compile the model
    model.compile("adam", loss=[loss_fn, None],)  # No loss and optimization based on word embeddings from transformer block
    
    return model


<br>

### DOWNLOAD AND PREPARE DATASET

---

Download the IMDB dataset and combine training and validation sets for a text
generation task.

In [8]:
# Download the dataset
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0 80.2M    0 32768    0     0  14546      0  1:36:23  0:00:02  1:36:21 14557
  0 80.2M    0  432k    0     0   131k      0  0:10:25  0:00:03  0:10:22  131k
  4 80.2M    4 3440k    0     0   819k      0  0:01:40  0:00:04  0:01:36  819k
 15 80.2M   15 12.7M    0     0  2515k      0  0:00:32  0:00:05  0:00:27 2626k
 28 80.2M   28 22.9M    0     0  3782k      0  0:00:21  0:00:06  0:00:15 4692k
 42 80.2M   42 34.3M    0     0  4892k      0  0:00:16  0:00:07  0:00:09 7113k
 56 80.2M   56 45.4M    0     0  5672k      0  0:00:14  0:00:08  0:00:06 9380k
 72 80.2M   72 57.8M    0     0  6440k      0  0:00

In [12]:
# Batch size
batch_size = 128

In [13]:
# Initialize a list for filesnames
filenames = []

# Loop over the directories
for dir in ["aclImdb/train/pos", "aclImdb/train/neg", "aclImdb/test/pos", "aclImdb/test/neg"]:
    
    # Loop over the files inside each directory
    for f in os.listdir(dir):
        
        # Append the filename to the list
        filenames.append(os.path.join(dir, f))

# Report
print(f"{len(filenames)} files")

15889 files


In [14]:
# Shuffle the filenames
random.shuffle(filenames)

# Load the dataset through tf.data
text_ds = tf.data.TextLineDataset(filenames)

# Shuffle the dataset
text_ds = text_ds.shuffle(buffer_size=256)

# Set the batch size
text_ds = text_ds.batch(batch_size)

In [15]:
# Function for custom standardization
def custom_standardization(input_string):
    
    # Lowercase the text
    lowercased = tf.strings.lower(input_string)
    
    # Remove html line-break tags
    stripped_html = tf.strings.regex_replace(lowercased, "<br />", " ")
    
    # Handle punctuation
    out = tf.strings.regex_replace(stripped_html, f"([{string.punctuation}])", r" \1")
    
    return out


# Text vectorization layer
vectorize_layer = tf.keras.layers.TextVectorization(standardize=custom_standardization,
                                                    max_tokens=vocab_size - 1,
                                                    output_mode="int",
                                                    output_sequence_length=maxlen + 1,
)

# Adapt the vectorization layer to the text
vectorize_layer.adapt(text_ds)

# Vocabulary list
vocab = vectorize_layer.get_vocabulary()  


In [17]:
# Function for preparing the inputs and labels
def prepare_lm_inputs_labels(text):
    
    # Add the extra dimension to the text
    text = tf.expand_dims(text, -1)
    
    # Vectorize the text
    tokenized_sentences = vectorize_layer(text)
    
    # Inputs (all words except the last)
    x = tokenized_sentences[:, :-1]
    
    # Labels (shifted one position)
    y = tokenized_sentences[:, 1:]
    
    return x, y

# Map the function to the dataset
text_ds = text_ds.map(prepare_lm_inputs_labels)

# Prefetch the dataset
text_ds = text_ds.prefetch(tf.data.AUTOTUNE)

TypeError: in user code:


    TypeError: outer_factory.<locals>.inner_factory.<locals>.tf__prepare_lm_inputs_labels() takes 1 positional argument but 2 were given


<br>

### CALLBACK FUNCTION

---

In [16]:
# Class for generating text
class TextGenerator(tf.keras.callbacks.Callback):
    """A callback to generate text from a trained model.
    1. Feed some starting prompt to the model
    2. Predict probabilities for the next token
    3. Sample the next token and add it to the next input

    Arguments:
        max_tokens: Integer, the number of tokens to be generated after prompt.
        start_tokens: List of integers, the token indices for the starting prompt.
        index_to_word: List of strings, obtained from the TextVectorization layer.
        top_k: Integer, sample from the `top_k` token predictions.
        print_every: Integer, print after this many epochs.
    """

    # Constructor function
    def __init__(self, max_tokens, start_tokens, index_to_word, top_k=10, print_every=1):
        
        # Initialization
        self.max_tokens = max_tokens
        self.start_tokens = start_tokens
        self.index_to_word = index_to_word
        self.print_every = print_every
        self.k = top_k

    # Function for sampling from the model
    def sample_from(self, logits):
        
        # Finds values and indices of the k largest entries for the last dimension.
        logits, indices = tf.math.top_k(logits, k=self.k, sorted=True)
        
        # Covert indices to numpy array
        indices = np.asarray(indices).astype("int32")
        
        # Softmax to convert logits to probabilities
        preds = tf.keras.activations.softmax(tf.expand_dims(logits, 0))[0]
        
        # Convert to numpy array
        preds = np.asarray(preds).astype("float32")
        
        # Generates a random sample from a given 1-D array
        out = np.random.choice(indices, p=preds)
        
        return out 

    # Function for converting indices to tokens
    def detokenize(self, number):
        
        # Convert index to word
        return self.index_to_word[number]

    # Function for generating text
    def on_epoch_end(self, epoch, logs=None):
        
        # Initialize the start tokens 
        start_tokens = [_ for _ in self.start_tokens]
        
        # Every `print_every` epochs
        if (epoch + 1) % self.print_every != 0:
            
            # Return
            return
        
        # Initialize the number of tokens generated
        num_tokens_generated = 0
        
        # Initialize the tokens generated
        tokens_generated = []
        
        # Loop until the number of tokens generated is less than the maximum number of tokens
        while num_tokens_generated <= self.max_tokens:
            
            # Pad length 
            pad_len = maxlen - len(start_tokens)
            
            # The index of the last token in the start_tokens
            sample_index = len(start_tokens) - 1
            
            # If the pad length is less than 0
            if pad_len < 0:
                
                # Inputs: Start tokens from 0 to maxlen
                x = start_tokens[:maxlen]
                
                # Set the sample index to maxlen - 1
                sample_index = maxlen - 1
                
            # If the pad length is greater than 0
            elif pad_len > 0:
                
                # Inputs: Start tokens and pad with 0s
                x = start_tokens + [0] * pad_len
                
            # If the pad length is 0
            else:
                
                # Inputs: Start tokens
                x = start_tokens
                
            # Convert to numpy array
            x = np.array([x])
            
            # Predict
            y, _ = self.model.predict(x)
            
            # Sample from the model
            sample_token = self.sample_from(y[0][sample_index])
            
            # Append the sample token to the generated tokens
            tokens_generated.append(sample_token)
            
            # Append the sample token to the start tokens
            start_tokens.append(sample_token)
            
            # Increment the number of tokens generated
            num_tokens_generated = len(tokens_generated)
            
        # Join the predicted tokens 
        txt = " ".join([self.detokenize(_) for _ in self.start_tokens + tokens_generated])
        
        # Report
        print(f"GENERATED TEXT:\n{txt}\n")

In [17]:
# Initialize a word2index dictionary
word_to_index = {}

# Loop over the vocabulary
for index, word in enumerate(vocab):
    
    # Add word and index to the dictionary
    word_to_index[word] = index

# Start prompt
start_prompt = "the police "

# Convert the start prompt to token indices
start_tokens = [word_to_index.get(_, 1) for _ in start_prompt.split()]

# Number of tokens to be generated
num_tokens_generated = 50

# Initialize the text generator
text_gen_callback = TextGenerator(num_tokens_generated, start_tokens, vocab)

<br>

### TRAINING

---

Note: This code should preferably be run on GPU.

In [18]:
# Initialize the model
model = create_model()

# Model summary
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 80)]              0         
_________________________________________________________________
token_and_position_embedding (None, 80, 256)           5140480   
_________________________________________________________________
transformer_block (Transform (None, 80, 256)           658688    
_________________________________________________________________
dense_2 (Dense)              (None, 80, 20000)         5140000   
Total params: 10,939,168
Trainable params: 10,939,168
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Train the model
model.fit(text_ds, verbose=1, epochs=1, callbacks=[text_gen_callback])

391/391 [==============================] - 39s 93ms/step - loss: 5.5724 - dense_2_loss: 5.5724
GENERATED TEXT:
the police , is a little bit too well , and the only the worst movie i have ever seen . this movie is a very good movie that i had to find out of the [UNK] the same time and i was in my opinion , but when i am going to



<br>

### PREDICTION

---

In [ ]:
# # Class for generating text
# class TextGenerator():

#     # Constructor function
#     def __init__(self, max_tokens, start_tokens, index_to_word, top_k=10, print_every=1):
        
#         # Initialization
#         self.max_tokens = max_tokens
#         self.start_tokens = start_tokens
#         self.index_to_word = index_to_word
#         self.print_every = print_every
#         self.k = top_k

#     # Function for sampling from the model
#     def sample_from(self, logits):
        
#         # Finds values and indices of the k largest entries for the last dimension.
#         logits, indices = tf.math.top_k(logits, k=self.k, sorted=True)
        
#         # Covert indices to numpy array
#         indices = np.asarray(indices).astype("int32")
        
#         # Softmax to convert logits to probabilities
#         preds = tf.keras.activations.softmax(tf.expand_dims(logits, 0))[0]
        
#         # Convert to numpy array
#         preds = np.asarray(preds).astype("float32")
        
#         # Generates a random sample from a given 1-D array
#         out = np.random.choice(indices, p=preds)
        
#         return out 

#     # Function for converting indices to tokens
#     def detokenize(self, number):
        
#         # Convert index to word
#         return self.index_to_word[number]

#     # Function for generating text
#     def on_epoch_end(self, epoch, logs=None):
        
#         # Initialize the start tokens 
#         start_tokens = [_ for _ in self.start_tokens]
        
#         # Every `print_every` epochs
#         if (epoch + 1) % self.print_every != 0:
            
#             # Return
#             return
        
#         # Initialize the number of tokens generated
#         num_tokens_generated = 0
        
#         # Initialize the tokens generated
#         tokens_generated = []
        
#         # Loop until the number of tokens generated is less than the maximum number of tokens
#         while num_tokens_generated <= self.max_tokens:
            
#             # Pad length 
#             pad_len = maxlen - len(start_tokens)
            
#             # The index of the last token in the start_tokens
#             sample_index = len(start_tokens) - 1
            
#             # If the pad length is less than 0
#             if pad_len < 0:
                
#                 # Inputs: Start tokens from 0 to maxlen
#                 x = start_tokens[:maxlen]
                
#                 # Set the sample index to maxlen - 1
#                 sample_index = maxlen - 1
                
#             # If the pad length is greater than 0
#             elif pad_len > 0:
                
#                 # Inputs: Start tokens and pad with 0s
#                 x = start_tokens + [0] * pad_len
                
#             # If the pad length is 0
#             else:
                
#                 # Inputs: Start tokens
#                 x = start_tokens
                
#             # Convert to numpy array
#             x = np.array([x])
            
#             # Predict
#             y, _ = self.model.predict(x)
            
#             # Sample from the model
#             sample_token = self.sample_from(y[0][sample_index])
            
#             # Append the sample token to the generated tokens
#             tokens_generated.append(sample_token)
            
#             # Append the sample token to the start tokens
#             start_tokens.append(sample_token)
            
#             # Increment the number of tokens generated
#             num_tokens_generated = len(tokens_generated)
            
#         # Join the predicted tokens 
#         txt = " ".join([self.detokenize(_) for _ in self.start_tokens + tokens_generated])
        
#         # Report
#         print(f"GENERATED TEXT:\n{txt}\n")

# # Initialize a word2index dictionary
# word_to_index = {}

# # Loop over the vocabulary
# for index, word in enumerate(vocab):
    
#     # Add word and index to the dictionary
#     word_to_index[word] = index

# # Start prompt
# start_prompt = "the wizard "

# # Convert the start prompt to token indices
# start_tokens = [word_to_index.get(_, 1) for _ in start_prompt.split()]

# # Number of tokens to be generated
# num_tokens_generated = 50

# # Initialize the text generator
# text_gen_callback = TextGenerator(num_tokens_generated, start_tokens, vocab)

<br>

### EVALUATION

---

In [ ]:
# TODO

In [3]:
import tensorflow as tf

def causal_attention_mask(batch_size, dest_seq_len, src_seq_len, dtype):
    i, j = tf.range(dest_seq_len)[:, None], tf.range(src_seq_len)
    mask = i >= (j - src_seq_len + dest_seq_len)
    mask = tf.cast(mask, dtype)
    mask = tf.reshape(mask, [1, dest_seq_len, src_seq_len])
    matrix_mult = tf.concat([tf.expand_dims(batch_size), tf.constant([1, 1], dtype=tf.int32)], axis=0)
    out = tf.tile(mask, matrix_mult)
    return out

class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, feed_forward_dim, dropout_rate):
        super(TransformerBlock, self).__init__()
        self.multi_head_attention = tf.keras.layers.MultiHeadAttention(num_heads, embed_dim)
        self.feed_forward_network = tf.keras.Sequential([
            tf.keras.layers.Dense(feed_forward_dim, activation="relu"),
            tf.keras.layers.Dense(embed_dim)
        ])
        self.layer_norm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layer_norm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(dropout_rate)
        self.dropout2 = tf.keras.layers.Dropout(dropout_rate)

    def call(self, inputs, training):
        input_shape = tf.shape(inputs)
        batch_size = input_shape[0]
        seq_len = input_shape[1]
        causal_mask = causal_attention_mask(batch_size, seq_len, seq_len, tf.bool, batch_size=batch_size)
        attention_output = self.multi_head_attention(inputs, inputs, attention_mask=causal_mask)
        attention_output = self.dropout1(attention_output, training=training)
        out1 = self.layer_norm1(inputs + attention_output)
        ffn_output = self.feed_forward_network(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        out = self.layer_norm2(out1 + ffn_output)
        return out

class TokenAndPositionEmbedding(tf.keras.layers.Layer):
    def __init__(self, max_seq_len, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_embedding = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.position_embedding = tf.keras.layers.Embedding(input_dim=max_seq_len, output_dim=embed_dim)

    def call(self, x):
        max_seq_len = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=max_seq_len, delta=1)
        positions = self.position_embedding(positions)
        x = self.token_embedding(x)
        out = x + positions
        return out

vocab_size = 20000
max_seq_len = 80
embed_dim = 256
num_heads = 2
feed_forward_dim = 256
dropout_rate = 0.1

def create_transformer_model():
    inputs = tf.keras.layers.Input(shape=(max_seq_len, ), dtype=tf.int32)
    embedding_layer = TokenAndPositionEmbedding(max_seq_len, vocab_size, embed_dim)
    x = embedding_layer(inputs)
    transformer_block = TransformerBlock(embed_dim, num_heads, feed_forward_dim, dropout_rate)
    x = transformer_block(x)
    outputs = tf.keras.layers.Dense(vocab_size)(x)
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    model = tf.keras.Model(inputs=inputs, outputs=[outputs, x])
    model.compile("adam", loss=[loss_fn, None])
    return model

# Initialize the model
model = create_transformer_model()
print(model.summary())

TypeError: Exception encountered when calling layer "transformer_block_1" (type TransformerBlock).

in user code:

    File "C:\Users\Soheil\AppData\Local\Temp\ipykernel_15100\3558770377.py", line 29, in call  *
        causal_mask = causal_attention_mask(batch_size, seq_len, seq_len, tf.bool, batch_size=batch_size)

    TypeError: outer_factory.<locals>.inner_factory.<locals>.tf__causal_attention_mask() got multiple values for argument 'batch_size'


Call arguments received by layer "transformer_block_1" (type TransformerBlock):
  • inputs=tf.Tensor(shape=(None, 80, 256), dtype=float32)
  • training=None